[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-0/basics.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/56295530-getting-set-up-video-guide)

# LangChain Academy

Welcome to LangChain Academy! 

## Context

At LangChain, we aim to make it easy to build LLM applications. One type of LLM application you can build is an agent. There’s a lot of excitement around building agents because they can automate a wide range of tasks that were previously impossible. 

In practice though, it is incredibly difficult to build systems that reliably execute on these tasks. As we’ve worked with our users to put agents into production, we’ve learned that more control is often necessary. You might need an agent to always call a specific tool first or use different prompts based on its state. 

To tackle this problem, we’ve built [LangGraph](https://langchain-ai.github.io/langgraph/) — a framework for building agent and multi-agent applications. Separate from the LangChain package, LangGraph’s core design philosophy is to help developers add better precision and control into agent workflows, suitable for the complexity of real-world systems.

## Course Structure

The course is structured as a set of modules, with each module focused on a particular theme related to LangGraph. You will see a folder for each module, which contains a series of notebooks. A video will accompany each notebook to help walk through the concepts, but the notebooks are also stand-alone, meaning that they contain explanations and can be viewed independently of the videos. Each module folder also contains a `studio` folder, which contains a set of graphs that can be loaded into [LangGraph Studio](https://github.com/langchain-ai/langgraph-studio), our IDE for building LangGraph applications.

## Setup

Before you begin, please follow the instructions in the `README` to create an environment and install dependencies.

## Chat models

In this course, we'll be using [Chat Models](https://python.langchain.com/v0.2/docs/concepts/#chat-models), which do a few things take a sequence of messages as inputs and return chat messages as outputs. LangChain does not host any Chat Models, rather we rely on third party integrations. [Here](https://python.langchain.com/v0.2/docs/integrations/chat/) is a list of 3rd party chat model integrations within LangChain! By default, the course will use [ChatOpenAI](https://python.langchain.com/v0.2/docs/integrations/chat/openai/) because it is both popular and performant. As noted, please ensure that you have an `OPENAI_API_KEY`.

Let's check that your `OPENAI_API_KEY` is set and, if not, you will be asked to enter it.

In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Nenhuma GPU CUDA disponível")

True
1
NVIDIA GeForce RTX 3060


In [2]:
#%%capture --no-stderr
#%pip install --quiet -U langchain_openai langchain_core langchain_community tavily-python
from langchain_community.llms import Ollama


In [3]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

#_set_env("OPENAI_API_KEY")

[Here](https://python.langchain.com/v0.2/docs/how_to/#chat-models) is a useful how-to for all the things that you can do with chat models, but we'll show a few highlights below. If you've run `pip install -r requirements.txt` as noted in the README, then you've installed the `langchain-openai` package. With this, we can instantiate our `ChatOpenAI` model object. If you are signing up for the API for the first time, you should receive [free credits](https://community.openai.com/t/understanding-api-limits-and-free-tier/498517) that can be applied to any of the models. You can see pricing for various models [here](https://openai.com/api/pricing/). The notebooks will default to `gpt-4o` because it's a good balance of quality, price, and speed [see more here](https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4-gpt-4-turbo-gpt-4o-and-gpt-4o-mini), but you can also opt for the lower priced `gpt-3.5` series models. 

There are [a few standard parameters](https://python.langchain.com/v0.2/docs/concepts/#chat-models) that we can set with chat models. Two of the most common are:

* `model`: the name of the model
* `temperature`: the sampling temperature

`Temperature` controls the randomness or creativity of the model's output where low temperature (close to 0) is more deterministic and focused outputs. This is good for tasks requiring accuracy or factual responses. High temperature (close to 1) is good for creative tasks or generating varied responses. 

In [4]:
from langchain_openai import ChatOpenAI
#gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)
#gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

Chat models in LangChain have a number of [default methods](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface). For the most part, we'll be using:

* `stream`: stream back chunks of the response
* `invoke`: call the chain on an input

And, as mentioned, chat models take [messages](https://python.langchain.com/v0.2/docs/concepts/#messages) as input. Messages have a role (that describes who is saying the message) and a content property. We'll be talking a lot more about this later, but here let's just show the basics.

In [5]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Lance")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
#gpt35_chat.invoke(messages)

We get an `AIMessage` response. Also, note that we can just invoke a chat model with a string. When a string is passed in as input, it is converted to a `HumanMessage` and then passed to the underlying model.


In [5]:
from langchain_community.llms import Ollama
llm = Ollama(
    model="llama3",
    base_url="http://localhost:11434",
    temperature=0.7
)
#response = llm.invoke(messages)
#print(response)

/tmp/ipykernel_3072/2360961892.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(


In [6]:
llm = Ollama(
    model="qwen2.5:14b", 
    base_url="http://localhost:11434",
    temperature=1.0
)
response = llm.invoke(messages)
print(response)

Hello! It's nice to meet you. How can I assist you today?


In [6]:
llm = Ollama(
    model="qwen2.5:14b", 
    base_url="http://localhost:11434",
    temperature=0.1
)
response = llm.invoke(messages)
print(response)

/tmp/ipykernel_4244/2094665964.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(


Hello! It's nice to meet you. How can I assist you today?


In [4]:
from langchain_community.llms import Ollama
llm = Ollama(
    model="qwen2.5:14b", 
    base_url="http://localhost:11434",
    temperature=0.7
)
#response = llm.invoke("用简单的术语解释量子计算")
#print(response)

/tmp/ipykernel_50744/3411072553.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(


In [7]:
llm = Ollama(
    model="qwen2.5:14b", 
    base_url="http://localhost:11434",
    temperature=0.2
)
response = llm.invoke("voce consegue guardar informações?")
print(response)

Como assistente de inteligência artificial, eu não armazeno informações pessoais ou específicas sobre as conversas que tenho com os usuários. Cada interação é tratada independentemente e privacidade dos usuários é uma prioridade para mim. No entanto, durante nossa sessão atual, posso lembrar das informações relevantes necessárias para fornecer assistência eficaz. Se você tiver alguma informação específica que gostaria de manter em mente durante nossa conversa, por favor me informe e farei o meu melhor para ajudar com base nessa informação.


In [8]:
llm = Ollama(
    model="qwen2.5:14b", 
    base_url="http://localhost:11434",
    temperature=0.2
)
response = llm.invoke("""
Preciso que voce guarde essas informações para desenvolvimento fututo:
Para lembre-se de colocar o install de todas as bibliotecas que forem ser utilizadas,não se esqueça de verficar se a rede neural
vai suportar, para evitar isso redimensione a imagem para 256x256
eu possuo uma 3060 com 12gb de Gram e meu computador é um I5 8600k com 6threads meu computador possui 32gb de ram
""")
print(response)


Claro, vou anotar essas informações para futuras consultas:

- Certifique-se de instalar todas as bibliotecas necessárias antes de começar o desenvolvimento.
- Verifique a compatibilidade da rede neural com os recursos disponíveis no seu hardware (GPU e CPU).
- Para evitar problemas de capacidade de processamento, redimensione as imagens para 256x256 pixels.
- Você possui uma GPU NVIDIA RTX 3060 com 12GB de VRAM.
- Seu computador é um processador Intel Core i5 8600K com 6 núcleos (threads).
- O sistema tem 32GB de memória RAM.

Essas informações serão úteis para orientar o desenvolvimento e garantir que os recursos do seu hardware sejam utilizados da melhor maneira possível. Se precisar de ajuda específica com algum aspecto desses detalhes, estou à disposição!


In [9]:
response = llm.invoke("""
eu sei tambem que a geração completa do código Python diretamente aqui seria extensa e complexa demais para ser gerada em tempo real 
por este assistente ma quero que voce faça mesmo assim código completo em Python usando PyTorch para treinar uma rede neural siamesa
para classificação de similaridade de imagens, com as seguintes exigências:
1. Utilize o dataset CIFAR-10 do torchvision.
2. Construa um dataset de **pares de imagens** (pares positivos e negativos) para treinar o modelo siamesa.
3. Modele uma **rede siamesa simples**, com backbone CNN compartilhado e camada de embedding.
4. Utilize **contrastive loss** como função de perda.
5. Aplique **validação cruzada estratificada (k=5)** nos pares.
6. Faça **seleção de hiperparâmetros** para:
   - Taxa de aprendizado ([0.01, 0.001])
   - Dimensão do embedding ([64, 128])
   - Número de filtros na primeira camada ([16, 32])
7. Para cada fold, calcule:
   - Acurácia de predição de similaridade
   - F1-Score (macro)
8. Ao final dos folds:
   - Calcule médias e desvios padrão das métricas
   - Execute um **teste t de Student pareado** entre os dois melhores conjuntos de hiperparâmetros
9. Gere uma **análise textual automática** dos resultados: qual configuração teve melhor desempenho, se a diferença é estatisticamente significativa etc.
10. Use apenas PyTorch, torchvision, scikit-learn e scipy.
11. Adicione comentários detalhados no código explicando cada parte.
12. O código deve ser completo, executável e sem textos fora de #comentarios.
""")
response = llm.invoke(response)
print(response)

Este script Python implementa uma abordagem completa para treinar um modelo Siamese Network usando o dataset CIFAR-10 com PyTorch. Ele inclui várias etapas importantes:

1. **Criação de Pares**: O script cria pares de imagens positivos e negativos a partir do conjunto de dados CIFAR-10, que é essencial para treinar uma rede Siamese.

2. **Modelo Siamese Network**: Define um modelo Siamese simples com duas redes idênticas (embedding_net) que compartilham pesos. O modelo é capaz de receber dois tensores como entrada e retornar os embeddings correspondentes.

3. **Contrastive Loss**: Implementa a função de perda contrastiva, que é crucial para o treinamento da rede Siamese. Esta função calcula a distância entre os embeddings dos pares de imagens e ajusta essa distância com base na similaridade (positivo ou negativo) do par.

4. **Treino e Avaliação**: O script inclui funções para treinar e avaliar o modelo, usando validação cruzada estratificada (k=5). Isso ajuda a obter uma estimativa ma

In [10]:
llm = Ollama(
    model="qwen2.5:14b", 
    base_url="http://localhost:11434",
    temperature=1.0
)

In [11]:
response = llm.invoke("""
eu sei tambem que a geração completa do código Python diretamente aqui seria extensa e complexa demais para ser gerada em tempo real 
por este assistente ma quero que voce faça mesmo assim código completo em Python usando PyTorch para treinar uma rede neural siamesa
para classificação de similaridade de imagens, com as seguintes exigências:
1. Utilize o dataset CIFAR-10 do torchvision.
2. Construa um dataset de **pares de imagens** (pares positivos e negativos) para treinar o modelo siamesa.
3. Modele uma **rede siamesa simples**, com backbone CNN compartilhado e camada de embedding.
4. Utilize **contrastive loss** como função de perda.
5. Aplique **validação cruzada estratificada (k=5)** nos pares.
6. Faça **seleção de hiperparâmetros** para:
   - Taxa de aprendizado ([0.01, 0.001])
   - Dimensão do embedding ([64, 128])
   - Número de filtros na primeira camada ([16, 32])
7. Para cada fold, calcule:
   - Acurácia de predição de similaridade
   - F1-Score (macro)
8. Ao final dos folds:
   - Calcule médias e desvios padrão das métricas
   - Execute um **teste t de Student pareado** entre os dois melhores conjuntos de hiperparâmetros
9. Gere uma **análise textual automática** dos resultados: qual configuração teve melhor desempenho, se a diferença é estatisticamente significativa etc.
10. Use apenas PyTorch, torchvision, scikit-learn e scipy.
11. Adicione comentários detalhados no código explicando cada parte.
12. O código deve ser completo, executável e sem textos fora de #comentarios.
""")
response = llm.invoke(response)
print(response)

O código fornecido é bem organizado para realizar a tarefa de encontrar o melhor conjunto de hiperparâmetros para um modelo siamesa baseado no dataset CIFAR-10. No entanto, há algumas áreas que precisam ser melhoradas ou ajustadas para garantir sua funcionalidade total:

### Correções e Melhorias

1. **Carregamento dos Dados**: O código atualmente carrega os dados do CIFAR-10 mas não cria DataLoaders para eles. É necessário criar `train_loader` e `val_loader` que serão usados durante o treinamento e teste.

2. **Criar Pares de Imagens**: A função `create_pairs()` está mal implementada. A versão atual gera uma quantidade arbitrária de pares, não considerando os índices do conjunto de dados e nem garantindo um equilíbrio entre classes diferentes (positivos e negativos).

3. **Treinamento no KFold**: O código realiza a validação cruzada `StratifiedKFold` mas não separa o conjunto de treino em conjuntos de treino e validação para cada fold.

4. **Comparação de Configurações com Teste t Par

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor, Resize
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score
import scipy.stats as st
from PIL import Image

# Transformação de dados
transform = Resize((256, 256))

class SiameseNetwork(nn.Module):
    def __init__(self, embedding_dim=4096, n_filters=32):
        super(SiameseNetwork, self).__init__()
        # CNN backbone compartilhado
        self.cnn = nn.Sequential(
            nn.Conv2d(1, n_filters, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(n_filters, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        # Camada de embedding
        self.embedding = nn.Sequential(
            nn.Linear(1024, embedding_dim),
            nn.ReLU()
        )

    def forward_one(self, x):
        output = self.cnn(x)
        output = output.view(output.size()[0], -1)  # Flattening
        output = self.embedding(output)
        return output

    def forward(self, input1, input2):
        return self.forward_one(input1), self.forward_one(input2)

def contrastive_loss(embedding_a, embedding_b, label, margin=1.0):
    distance = torch.pairwise_distance(embedding_a, embedding_b)

    loss_contrastive = torch.mean((label) * torch.pow(distance, 2) +
                                    (1 - label) * torch.pow(torch.clamp(margin - distance, min=0.0), 2))
    return loss_contrastive

class SiameseDataset(Dataset):
    def __init__(self, mnist_dataset, transform=None):
        self.mnist = mnist_dataset
        self.transform = transform

    def __getitem__(self, index):
        img1, label1 = self.mnist[index]

        # Seleciona uma segunda imagem com a mesma classe ou diferente de acordo com o rótulo desejado.
        if torch.rand(1) > 0.5:
            similar_label = True
            label2 = label1
        else:
            similar_label = False
            while True:
                index2 = torch.randint(len(self.mnist), (1,))
                img2, label2 = self.mnist[index2]
                if label2 != label1: break

        # Transformação de imagens para 256x256
        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return (img1, img2), int(similar_label)

    def __len__(self):
        return len(self.mnist)

def train_and_validate_siamese(model, dataloader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for (img1, img2), label in dataloader:
            img1, img2, label = img1.to(device).unsqueeze(1), img2.to(device).unsqueeze(1), torch.tensor([label]).to(device)

            optimizer.zero_grad()
            embedding_a, embedding_b = model(img1.float(), img2.float())
            loss = criterion(embedding_a, embedding_b, label)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * len(label)

        print(f'Epoch {epoch + 1}/{epochs} - Loss: {running_loss / len(dataloader.dataset):.4f}')

def evaluate_siamese(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for (img1, img2), label in dataloader:
            img1, img2, label = img1.to(device).unsqueeze(1), img2.to(device).unsqueeze(1), torch.tensor([label]).to(device)

            embedding_a, embedding_b = model(img1.float(), img2.float())
            distance = torch.pairwise_distance(embedding_a, embedding_b)

            pred = (distance < 0.5).int().cpu()
            all_labels.extend(label.cpu().numpy())
            all_preds.extend(pred.numpy())

    return accuracy_score(all_labels, all_preds), f1_score(all_labels, all_preds, average='macro')

# Configuração do dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Carregamento dos dados MNIST
train_dataset = MNIST(root='./data', train=True, download=True, transform=ToTensor())
test_dataset = MNIST(root='./data', train=False, download=True, transform=ToTensor())

# Redimensionamento das imagens para 256x256
train_dataset.transform = transform
test_dataset.transform = transform

# Criação do dataset personalizado e carregamento em DataLoader
siamese_train_dataset = SiameseDataset(train_dataset)
siamese_test_dataset = SiameseDataset(test_dataset)

train_loader = DataLoader(siamese_train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(siamese_test_dataset, batch_size=64, shuffle=False)

# Inicialização do modelo e otimizador
model = SiameseNetwork().to(device)
criterion = contrastive_loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Treinamento e validação com k-fold cross-validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True)

for fold, (train_idx, val_idx) in enumerate(kf.split(siamese_train_dataset)):
    print(f'Fold {fold + 1}/{k_folds}')

    train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
    valid_sampler = torch.utils.data.SubsetRandomSampler(val_idx)

    train_loader_fold = DataLoader(siamese_train_dataset, batch_size=64, sampler=train_sampler)
    val_loader_fold = DataLoader(siamese_train_dataset, batch_size=64, sampler=valid_sampler)

    model = SiameseNetwork().to(device)  # Reinicializa o modelo para cada fold
    criterion = contrastive_loss
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_and_validate_siamese(model, train_loader_fold, criterion, optimizer, epochs=5)

# Avaliação final no conjunto de teste
accuracy, f1_score_ = evaluate_siamese(model, test_loader)
print(f'Test Accuracy: {accuracy:.4f}, F1 Score: {f1_score_:.4f}')

# Análise dos resultados e comparação de hiperparâmetros (exemplo simplificado)
best_model_params = {'lr': 0.001, 'epochs': 5}
second_best_model_params = {'lr': 0.0001, 'epochs': 10}

# Supondo que os resultados foram armazenados
results_best_model = (accuracy, f1_score_)
results_second_best_model = (accuracy - 0.02, f1_score_ - 0.01)

statistic, p_value = st.ttest_ind([results_best_model[1]], [results_second_best_model[1]])
print(f'p-value: {p_value}')
if p_value < 0.05:
    print('There is a statistically significant difference between the models.')
else:
    print('No statistically significant difference found.')

Fold 1/5


UnboundLocalError: cannot access local variable 'img2' where it is not associated with a value

In [30]:
response = llm.invoke("""
esse codigo não roda me ajude, me entregue um codigo que funciona

---------------------------------------------------------------------------
UnboundLocalError                         Traceback (most recent call last)
Cell In[28], line 52
     50 for epoch in range(5):  # Treinamento por 5 épocas
     51     model.train()
---> 52     for images, labels in train_loader_fold:
     53         images = images.to(device)
     54         labels = labels.float().to(device)  # Converter para float

File ~/anaconda3/envs/agentsLang/lib/python3.11/site-packages/torch/utils/data/dataloader.py:733, in _BaseDataLoaderIter.__next__(self)
    730 if self._sampler_iter is None:
    731     # TODO(https://github.com/pytorch/pytorch/issues/76750)
    732     self._reset()  # type: ignore[call-arg]
--> 733 data = self._next_data()
    734 self._num_yielded += 1
    735 if (
    736     self._dataset_kind == _DatasetKind.Iterable
    737     and self._IterableDataset_len_called is not None
    738     and self._num_yielded > self._IterableDataset_len_called
    739 ):

File ~/anaconda3/envs/agentsLang/lib/python3.11/site-packages/torch/utils/data/dataloader.py:789, in _SingleProcessDataLoaderIter._next_data(self)
    787 def _next_data(self):
    788     index = self._next_index()  # may raise StopIteration
--> 789     data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
    790     if self._pin_memory:
    791         data = _utils.pin_memory.pin_memory(data, self._pin_memory_device)

File ~/anaconda3/envs/agentsLang/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py:52, in _MapDatasetFetcher.fetch(self, possibly_batched_index)
     50         data = self.dataset.__getitems__(possibly_batched_index)
     51     else:
---> 52         data = [self.dataset[idx] for idx in possibly_batched_index]
     53 else:
     54     data = self.dataset[possibly_batched_index]

File ~/anaconda3/envs/agentsLang/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py:52, in <listcomp>(.0)
     50         data = self.dataset.__getitems__(possibly_batched_index)
     51     else:
---> 52         data = [self.dataset[idx] for idx in possibly_batched_index]
     53 else:
     54     data = self.dataset[possibly_batched_index]

Cell In[25], line 74, in SiameseDataset.__getitem__(self, index)
     71     img1 = self.transform(img1)
     72     img2 = self.transform(img2)
---> 74 return (img1, img2), int(similar_label)

UnboundLocalError: cannot access local variable 'img2' where it is not associated with a value


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor, Resize
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score
from scipy.stats import ttest_rel

# Transformação de dados
transform = Resize((256, 256))

class SiameseNetwork(nn.Module):
    def __init__(self, embedding_dim=4096, n_filters=32):
        super(SiameseNetwork, self).__init__()
        # CNN backbone compartilhado
        self.cnn = nn.Sequential(
            nn.Conv2d(1, n_filters, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(n_filters, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        
        # Camada de embedding
        self.embedding = nn.Sequential(
            nn.Linear(1024, embedding_dim), 
            nn.ReLU()
        )

    def forward_one(self, x):
        output = self.cnn(x)
        output = output.view(output.size()[0], -1)  # Flattening
        output = self.embedding(output)
        return output
    
    def forward(self, input1, input2):
        return self.forward_one(input1), self.forward_one(input2)

def contrastive_loss(embedding_a, embedding_b, label, margin=1.0):
    distance = torch.pairwise_distance(embedding_a, embedding_b)
    
    loss_contrastive = torch.mean((label) * torch.pow(distance, 2) +
                                   (1 - label) * torch.pow(torch.clamp(margin - distance, min=0.0), 2))
    return loss_contrastive

class SiameseDataset(Dataset):
    def __init__(self, mnist_dataset, transform=None):
        self.mnist = mnist_dataset
        self.transform = transform
        
    def __getitem__(self, index):
        img1, label1 = self.mnist[index]
        
        # Seleciona uma segunda imagem com a mesma classe ou diferente de acordo com o rótulo desejado.
        if torch.rand(1) > 0.5:
            similar_label = True
            label2 = label1
        else:
            similar_label = False
            while True:
                index2 = torch.randint(len(self.mnist), (1,))
                img2, label2 = self.mnist[index2]
                if label2 != label1: break
        
        # Transformação de imagens para 256x256
        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return (img1, img2), int(similar_label)

    def __len__(self):
        return len(self.mnist)

def train_and_validate_siamese(model, dataloader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for (img1, img2), label in dataloader:
            img1, img2, label = img1.to(device).unsqueeze(1), img2.to(device).unsqueeze(1), torch.tensor([label]).to(device)
            
            optimizer.zero_grad()
            embedding_a, embedding_b = model(img1.float(), img2.float())
            loss = criterion(embedding_a, embedding_b, label)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * len(label)

        print(f'Epoch {epoch + 1}/{epochs} - Loss: {running_loss / len(dataloader.dataset):.4f}')

def evaluate_siamese(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for (img1, img2), label in dataloader:
            img1, img2, label = img1.to(device).unsqueeze(1), img2.to(device).unsqueeze(1), torch.tensor([label]).to(device)
            
            embedding_a, embedding_b = model(img1.float(), img2.float())
            distance = torch.pairwise_distance(embedding_a, embedding_b)

            pred = (distance < 0.5).int().cpu()
            all_labels.extend(label.cpu().numpy())
            all_preds.extend(pred.numpy())

    return accuracy_score(all_labels, all_preds), f1_score(all_labels, all_preds, average='macro')

# Configuração do dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Carregamento dos dados MNIST
train_dataset = MNIST(root='./data', train=True, download=True, transform=ToTensor())
test_dataset = MNIST(root='./data', train=False, download=True, transform=ToTensor())

# Redimensionamento das imagens para 256x256
train_dataset.transform = transform
test_dataset.transform = transform

# Criação do dataset personalizado e carregamento em DataLoader
siamese_train_dataset = SiameseDataset(train_dataset)
siamese_test_dataset = SiameseDataset(test_dataset)

train_loader = DataLoader(siamese_train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(siamese_test_dataset, batch_size=64, shuffle=False)

# Inicialização do modelo e otimizador
model = SiameseNetwork().to(device)
criterion = contrastive_loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Treinamento e validação com k-fold cross-validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True)

for fold, (train_idx, val_idx) in enumerate(kf.split(siamese_train_dataset)):
    print(f'Fold {fold + 1}/{k_folds}')
    
    train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
    valid_sampler = torch.utils.data.SubsetRandomSampler(val_idx)

    train_loader_fold = DataLoader(siamese_train_dataset, batch_size=64, sampler=train_sampler)
    val_loader_fold = DataLoader(siamese_train_dataset, batch_size=64, sampler=valid_sampler)

    model = SiameseNetwork().to(device)  # Reinicializa o modelo para cada fold
    criterion = contrastive_loss
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    train_and_validate_siamese(model, train_loader_fold, criterion, optimizer, epochs=5)

# Avaliação final no conjunto de teste
accuracy, f1_score_ = evaluate_siamese(model, test_loader)
print(f'Test Accuracy: {accuracy:.4f}, F1 Score: {f1_score_:.4f}')

# Análise dos resultados e comparação de hiperparâmetros (exemplo simplificado)
best_model_params = {'lr': 0.001, 'epochs': 5}
second_best_model_params = {'lr': 0.0001, 'epochs': 10}

# Supondo que os resultados foram armazenados
results_best_model = (accuracy, f1_score_)
results_second_best_model = (accuracy - 0.02, f1_score_ - 0.01)

statistic, p_value = scipy.stats.ttest_ind(results_best_model[1], results_second_best_model[1])
print(f'p-value: {p_value}')
if p_value < 0.05:
    print('There is a statistically significant difference between the models.')
else:
    print('No statistically significant difference found.')
""")
print(response)

The error you encountered (`NameError: name 'distance' not defined`) in your `evaluate_siamese` function indicates that there's an issue with how the distance calculation and prediction logic are structured. Specifically, it seems like the variable `distance` is being used before it is properly computed.

Let's address this by ensuring that the distance computation and prediction steps are correctly implemented:

1. **Ensure Distance Calculation**: The pairwise distance between embeddings should be calculated.
2. **Threshold Prediction**: Use a threshold (e.g., 0.5) to convert distances into binary predictions.
3. **Collect Metrics**: Collect all labels and predictions for evaluation.

Here's the corrected `evaluate_siamese` function:

```python
def evaluate_siamese(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for (img1, img2), label in dataloader:
            img1, img2, label = img1.to(device).unsqueeze(1), img2.to(dev

In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score

In [4]:
def evaluate_siamese(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for (img1, img2), label in dataloader:
            img1, img2, label = img1.to(device).unsqueeze(1), img2.to(device).unsqueeze(1), torch.tensor([label]).to(device)

            embedding_a, embedding_b = model(img1.float(), img2.float())
            distance = torch.pairwise_distance(embedding_a, embedding_b)  # Ensure this line is correct

            pred = (distance < 0.5).int().cpu()  # Convert distances to binary predictions
            all_labels.extend(label.cpu().numpy())  # Collect labels
            all_preds.extend(pred.numpy())  # Collect predictions

    return accuracy_score(all_labels, all_preds), f1_score(all_labels, all_preds, average='macro')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data loading and preprocessing
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

# Custom dataset for Siamese network
class SiameseDataset:
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img1, label1 = self.dataset[idx]
        while True:
            img2, label2 = self.dataset[torch.randint(len(self.dataset), (1,)).item()]
            if label1 != label2:  # Ensure different labels
                break
        return img1, img2, int(label1 == label2)

# Custom dataset for Siamese network with resizing
train_dataset.transform = transforms.Compose([transforms.Resize((256, 256)), train_dataset.transform])
test_dataset.transform = transforms.Compose([transforms.Resize((256, 256)), test_dataset.transform])

siamese_train_dataset = SiameseDataset(train_dataset)
siamese_test_dataset = SiameseDataset(test_dataset)

train_loader = DataLoader(siamese_train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(siamese_test_dataset, batch_size=64, shuffle=False)

# Model and optimizer
class SiameseNetwork(torch.nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2)
        )
        # Add more layers as needed
        self.fc1 = torch.nn.Linear(32 * 61 * 61, 4096)

    def forward_one(self, x):
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        return output1, output2

model = SiameseNetwork().to(device)

def contrastive_loss(embedding_a, embedding_b, label):
    distance = torch.pairwise_distance(embedding_a, embedding_b)
    loss_contrastive = torch.mean(label * torch.pow(distance, 2) + (1 - label) * torch.pow(torch.clamp(1.0 - distance, min=0.0), 2))
    return loss_contrastive

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train_and_validate_siamese(model, dataloader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for img1, img2, label in dataloader:  # Modifiquei aqui para receber três valores
            img1, img2, label = img1.to(device).unsqueeze(1), img2.to(device).unsqueeze(1), label.to(device).float() # Convert label to float for loss calculation
            optimizer.zero_grad()
            embedding_a, embedding_b = model(img1.float(), img2.float())
            loss = criterion(embedding_a, embedding_b, label)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(dataloader)}")

def evaluate_siamese(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for img1, img2, label in dataloader: # Modifiquei aqui para receber três valores
            img1, img2, label = img1.to(device).unsqueeze(1), img2.to(device).unsqueeze(1), label.to(device)

            embedding_a, embedding_b = model(img1.float(), img2.float())
            distance = torch.pairwise_distance(embedding_a, embedding_b)

            pred = (distance < 0.5).int().cpu()
            all_labels.extend(label.cpu().numpy())
            all_preds.extend(pred.numpy())

    return accuracy_score(all_labels, all_preds), f1_score(all_labels, all_preds, average='macro')

# Training and evaluation loop with KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42) # Adicionei shuffle e random_state para reprodutibilidade
for fold, (train_idx, val_idx) in enumerate(kf.split(siamese_train_dataset)):
    print(f"Training Fold {fold + 1}")
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    train_loader_fold = DataLoader(siamese_train_dataset, batch_size=64, sampler=train_sampler)
    val_loader_fold = DataLoader(siamese_train_dataset, batch_size=64, sampler=val_sampler)

    model = SiameseNetwork().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    train_and_validate_siamese(model, train_loader_fold, contrastive_loss, optimizer, epochs=5)

    print(f"Evaluation Fold {fold + 1}")
    acc, f1 = evaluate_siamese(model, val_loader_fold)
    print(f"Accuracy: {acc}, F1 Score: {f1}")

# Final evaluation on test set
print("Final Evaluation")
acc, f1 = evaluate_siamese(model, test_loader)
print(f"Test Accuracy: {acc}, Test F1 Score: {f1}")

Training Fold 1


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [64, 1, 1, 256, 256]

In [6]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data loading and preprocessing
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

# Custom dataset for Siamese network
class SiameseDataset:
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img1, label1 = self.dataset[idx]
        while True:
            img2, label2 = self.dataset[torch.randint(len(self.dataset), (1,)).item()]
            if label1 != label2:  # Ensure different labels
                break
        return img1, img2, int(label1 == label2)

# Custom dataset for Siamese network with resizing
train_dataset.transform = transforms.Compose([transforms.Resize((256, 256)), train_dataset.transform])
test_dataset.transform = transforms.Compose([transforms.Resize((256, 256)), test_dataset.transform])

siamese_train_dataset = SiameseDataset(train_dataset)
siamese_test_dataset = SiameseDataset(test_dataset)

train_loader = DataLoader(siamese_train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(siamese_test_dataset, batch_size=64, shuffle=False)

# Model and optimizer
class SiameseNetwork(torch.nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2)
        )
        # Add more layers as needed
        self.fc1 = torch.nn.Linear(32 * 61 * 61, 4096)

    def forward_one(self, x):
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        return output1, output2

model = SiameseNetwork().to(device)

def contrastive_loss(embedding_a, embedding_b, label):
    distance = torch.pairwise_distance(embedding_a, embedding_b)
    loss_contrastive = torch.mean(label * torch.pow(distance, 2) + (1 - label) * torch.pow(torch.clamp(1.0 - distance, min=0.0), 2))
    return loss_contrastive

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train_and_validate_siamese(model, dataloader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for (img1, img2), label in dataloader:
            img1, img2, label = img1.to(device).unsqueeze(1), img2.to(device).unsqueeze(1), label.to(device)
            optimizer.zero_grad()
            embedding_a, embedding_b = model(img1.float(), img2.float())
            loss = criterion(embedding_a, embedding_b, label.float())
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(dataloader)}")

def evaluate_siamese(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for (img1, img2), label in dataloader:
            img1, img2, label = img1.to(device).unsqueeze(1), img2.to(device).unsqueeze(1), label.to(device)

            embedding_a, embedding_b = model(img1.float(), img2.float())
            distance = torch.pairwise_distance(embedding_a, embedding_b)  # Ensure this line is correct

            pred = (distance < 0.5).int().cpu()  # Convert distances to binary predictions
            all_labels.extend(label.cpu().numpy())  # Collect labels
            all_preds.extend(pred.numpy())  # Collect predictions

    return accuracy_score(all_labels, all_preds), f1_score(all_labels, all_preds, average='macro')

# Training and evaluation loop with KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42) # Adicionei shuffle e random_state para reprodutibilidade
for fold, (train_idx, val_idx) in enumerate(kf.split(siamese_train_dataset)):
    print(f"Training Fold {fold + 1}")
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    train_loader_fold = DataLoader(siamese_train_dataset, batch_size=64, sampler=train_sampler)
    val_loader_fold = DataLoader(siamese_train_dataset, batch_size=64, sampler=val_sampler)

    model = SiameseNetwork().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    train_and_validate_siamese(model, train_loader_fold, contrastive_loss, optimizer, epochs=5)

    print(f"Evaluation Fold {fold + 1}")
    acc, f1 = evaluate_siamese(model, val_loader_fold)
    print(f"Accuracy: {acc}, F1 Score: {f1}")

# Final evaluation on test set
print("Final Evaluation")
acc, f1 = evaluate_siamese(model, test_loader)
print(f"Test Accuracy: {acc}, Test F1 Score: {f1}")

Training Fold 1


ValueError: too many values to unpack (expected 2)